# M2 axial Actuator force limit

This Jupyter notebook is to run the M2 test for the software limit for the axial actuators
An axial actuator will be issued with a force over the software limit and check that the command will be rejected by the CSCejected by the CSC.

## Import Modules

In [1]:
import asyncio
from datetime import datetime
from lsst.ts import salobj
from lsst.ts.m2com import NUM_ACTUATOR, NUM_TANGENT_LINK

## Declaration of User-defined Functions

In [2]:
async def injectForce_axial_actuator(csc, actuators, force, sleep_time=5):
    """inject a force to the axial actuator.
    
    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the M2 CSC.
    actuators : list of actuators.
    force : list of forces
        Force to apply (Newton).
    sleep_time : float, optional
        Sleep time. (the default is 5.0)
    """

    num_axial_actuator = NUM_ACTUATOR - NUM_TANGENT_LINK
    forces = [0.] * num_axial_actuator
    
    for idx in range(len(actuators)):
        print(f"idx: {idx}.")
        index = actuators[idx]
        forces[index] = force
        print(f"Apply the force: {abs(force)} N. to actuator: {index}")
    await csc.cmd_applyForces.set_start(axial=forces)
    await asyncio.sleep(sleep_time)


## Prepare the M2 CSC and put to Enabled state

In [3]:
domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

In [ ]:
# get status

state = m2.evt_summaryState.get()
print(state)

In [ ]:
# Standby  -->  Disable
await m2.cmd_start.set_start(timeout=60)

In [ ]:
# Disable  -->  Enabled
await m2.cmd_enable.set_start(timeout=200)

In [ ]:
# Enabled  -->  Disable
await m2.cmd_disable.set_start(timeout=30)

In [ ]:
#Disable  -->  Standby
await m2.cmd_standby.set_start(timeout=30)

In [ ]:
#Fault --> Standby
await m2.cmd_standby.set_start(timeout=30)

## Do the force actuator limit Test

In [ ]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

In [8]:
# reset applied forces to axial actuators

num_axial_actuator = NUM_ACTUATOR - NUM_TANGENT_LINK
forces = [0.] * num_axial_actuator

for idx in range(num_axial_actuator):
    force = forces[idx]
await m2.cmd_resetForceOffsets.set_start()
await asyncio.sleep(5)

In [6]:
force = 45
actuators = [15]
await injectForce_axial_actuator(m2, actuators, force, sleep_time=5)

idx: 0.
Apply the force: 45 N. to actuator: 15


In [ ]:
force = 200
actuators = [15]
await injectForce_axial_actuator(m2, actuators, force, sleep_time=5)

In [ ]:
# command a force beyond the force limit that we expect to be rejected by the CSC 

force = 400
actuators = [15]
await injectForce_axial_actuator(m2, actuators, force, sleep_time=5)